In [17]:
%load_ext autoreload
%autoreload 2
%pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/workspace'

In [18]:
! pip uninstall cudams -q -y
! pip install git+https://github.com/tornikeo/cosine-similarity.git@dev

  Cloning https://github.com/tornikeo/cosine-similarity.git (to revision dev) to /tmp/pip-req-build-t2aisiwe
  Running command git clone --filter=blob:none --quiet https://github.com/tornikeo/cosine-similarity.git /tmp/pip-req-build-t2aisiwe
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  Branch 'dev' set up to track remote branch 'dev' from 'origin'.
  Resolved https://github.com/tornikeo/cosine-similarity.git to commit 305448299d2b9bdb372dee08fe1192618aff2ac4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cudams: filename=cudams-0.0.1-py3-none-any.whl size=18066 sha256=0b6efc4482fe0c6a1d1a1207cc443612594c5d5b1fcedd2a56ef34eb608d0a4c
  Stored in directory: /tmp/pip-ephem-wheel-cache-qs4m3qt5/wheels/69/5a/b2/042e17f5c3a426762af6e3eded8492596b447c5c6dd3dcfdd8
Successfully built cudams


# Load data

In [19]:
from cudams.utils import \
    argbatch, mkdir, get_ref_spectra_from_df
import math
from pathlib import Path
import pandas as pd
from itertools import product
from time import perf_counter
from multiprocessing.pool import ThreadPool
from multiprocessing import shared_memory
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import torch
import numba
from numba import cuda

assert torch.cuda.is_available()
assert cuda.is_available()

In [20]:
from cudams.similarity.kernels import compile_cuda_cosine_greedy_kernel

match_limit = 1024
max_peaks = 1024
batch_size = 2048
threshold = .75

kernel = compile_cuda_cosine_greedy_kernel(
    tolerance=.1,
    shift=0,
    mz_power=0,
    int_power=1,
    match_limit=match_limit,
    batch_size=batch_size,
)

In [27]:
from cudams.utils import download_cosine_100k_sample
from pathlib import Path

spectra_file = download_cosine_100k_sample('spectra_100k.csv')

In [28]:
! du -h *

63M	cosine.csv
4.0K	onstart.sh
4.0K	ports.log
16K	sparse_standalone.ipynb
60M	spectra_100k.csv
152K	tests/data
152K	tests
16K	tutorial_pesticide.ipynb


In [29]:
from cudams.utils import get_spectra_batches

references, queries, batched_inputs = get_spectra_batches(
    reference_csv_file=spectra_file, # File paths
    query_csv_file=spectra_file,
    batch_size=batch_size, # Batch size for processing - large batch size requires more powerful hardware, and is faster
    max_peaks=max_peaks, # Number of max peaks in each batch. Large number requires much more memory and time, but is more accurate (after 1024, it doesn't matter much)
    padding=None, # Not required for current kernels
    max_pairs=(batch_size**2) * 256 # we use 256 batches
)

100%|██████████| 65536/65536 [00:11<00:00, 5696.16it/s]


In [39]:
device = torch.device('cuda')
! rm -rf data/output
! mkdir -p data/output
with torch.no_grad():
    for batch_i in tqdm(range(len(batched_inputs))):
        (rspec, rlen, rstart, rend), (qspec, qlen, qstart, qend) = batched_inputs[
            batch_i
        ]
        
        lens = torch.zeros(2, batch_size, dtype=torch.int32)
        lens[0, :len(rlen)] = torch.from_numpy(rlen)
        lens[1, :len(qlen)] = torch.from_numpy(qlen)
        
        rspec = torch.from_numpy(rspec)
        qspec = torch.from_numpy(qspec)
        out = torch.zeros(3, batch_size, batch_size, dtype=torch.float32).to(device)
        
        rspec = cuda.as_cuda_array(rspec)
        qspec = cuda.as_cuda_array(qspec)
        lens = cuda.as_cuda_array(lens)
        out = cuda.as_cuda_array(out)
        
        kernel(rspec, qspec, lens, out)
        
        mask = out[0] >= threshold
        
        row, col = cp.nonzero(mask)
        score, matches, overflow = out[:, mask]
        rabs = rstart + row
        qabs = qstart + col
        cp.savez_compressed(f'data/output/{rstart}-{rend}-{qstart}-{qend}.npz', 
                            rabs=rabs, 
                            qabs=qabs, 
                            score=score, 
                            matches=matches,
                            overflow=overflow)

  0%|          | 0/256 [00:00<?, ?it/s]


TypeError: *obj* doesn't implement the cuda array interface.

In [77]:
! du -hs data/output

5,1M	data/output


In [82]:
# Suppose we want to query these absolute query IDs, and sort their results

query = np.array([1, 42, 121, 100010])


In [89]:
q = cp.asarray(query)
for file in Path('data/output').glob('*.npz'):
    bunch = cp.load(file)
    qabs = bunch['qabs']
    interest = cp.isin(q, qabs)
    if interest.any():
        # We need to accumulate interests over all batches 
        # Concat them, sort them by `score` and return
        # cp.where(qabs == , )
    print(interest)
    break

[False False False False]
